In [ ]:
import torch
import numpy as np
import torch.nn as nn
import random

torch.manual_seed(8000)
random.seed(8000)
np.random.seed(8000)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
real_max_agent = 34 # This's just an example

def getPositionEncoding(seq_len, d, n=10000):
    P = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            P[(k*real_max_agent):((k+1)*real_max_agent), 2*i] = np.sin(k/denominator)
            P[(k*real_max_agent):((k+1)*real_max_agent), 2*i+1] = np.cos(k/denominator)
    return P


class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert(self.head_dim*heads == embed_size) #ensure they're divisible

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.query = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(self.heads*self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        values = values.reshape(N, value_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)

        values = self.values(values)
        query = self.query(query)
        keys = self.keys(keys)

        energy = torch.einsum("nqhd,nkhd->nhqk",[query, keys])
        # # query shape: (N, query_len, heads, heads_dim), query_len, key_len = MaxLength
        # # key shape: (N, key_len, heads, heads_dim)
        # # energy shape: (N, heads, query_len, key_len)

        # attention = torch.softmax(energy, dim=3)

        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e20)

        attention = torch.softmax(energy / (self.embed_size ** (1/2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values])
        out = out.reshape(N, query_len, self.heads*self.head_dim)
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, head_dim), key_len always equal to value_len
        # out shape: (N, query_len, heads, head_dim), then flatten last two dimensions

        out = self.fc_out(out)

        return out

class AgentAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(AgentAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert(self.head_dim*heads == embed_size) #ensure they're divisible

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys_self = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys_other = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.query_self = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.query_other = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(self.heads*self.head_dim, embed_size)

    def forward(self, values, keys, query, mask, inverse_mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        values = values.reshape(N, value_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)

        values = self.values(values)
        query_self = self.query_self(query)
        query_other = self.query_other(query)
        keys_self = self.keys_self(keys)
        keys_other = self.keys_other(keys)

        energy_self = torch.einsum("nqhd,nkhd->nhqk",[query_self, keys_self])
        energy_other = torch.einsum("nqhd,nkhd->nhqk",[query_other, keys_other])
        # # query shape: (N, query_len, heads, heads_dim)
        # # key shape: (N, key_len, heads, heads_dim)
        # # energy shape: (N, heads, query_len, key_len)

        # attention = torch.softmax(energy, dim=3)

        if mask is not None:
            energy_self = energy_self.masked_fill(mask == 0, float("-1e20"))
            energy_other = energy_other.masked_fill(inverse_mask == 0, float("-1e20"))

        attention = torch.softmax((energy_self + energy_other) / (self.head_dim ** (1/2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values])
        out = out.reshape(N, query_len, self.heads*self.head_dim)
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, head_dim), key_len always equal to value_len
        # out shape: (N, query_len, heads, head_dim), then flatten last two dimensions

        out = self.fc_out(out)

        return out#, attention, energy_self, energy_other

class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        # self.attention = SelfAttention(embed_size, heads)
        self.attention = AgentAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion*embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion*embed_size, embed_size)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask, inverse_mask):
        attention = self.attention(value, key, query, mask, inverse_mask)

        x = self.dropout(self.norm1(attention + query)) # Add & Norm
        forward = self.feed_forward(x) # Feed Forward
        out = self.dropout(self.norm2(forward + x)) # Add & Norm

        return out

class Bert(nn.Module):
    def __init__(self, embed_size,
                 num_layers, heads, forward_expansion,
                 dropout, max_length):
        super(Bert, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.num_layers = num_layers
        self.embedding = nn.Linear(max_length,embed_size)
        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion
                ) for _ in range(num_layers)
            ]
        )
        self.final_out = nn.Linear(embed_size,max_length)
        self.relu = nn.ReLU()


    def forward(self, x, pos_encoding, mask, inverse_mask):
        N, seq_length, feature_length = x.shape
        out = self.embedding(x) + pos_encoding

        for layers in self.layers:
            out = layers(out,out,out,mask,inverse_mask)

        out = self.final_out(out)

        return out